In [1]:
# !pip3 install tensorflow==2.9.0 tensorflow-gpu
# !pip3 uninstall tensorflow --yes
# !pip3 uninstall tensorflow-gpu
# ! pip3 install tensorflow-gpu

In [2]:
!rm -rf *

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as random
from PIL import Image
import os
import datetime
import cv2
import glob
import json
from types import MethodType
from sklearn.metrics import classification_report, confusion_matrix, f1_score

import six

import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Softmax, Reshape, Input, Flatten, Conv2DTranspose, Conv2D, GlobalAveragePooling2D,BatchNormalization, Multiply, Dot, Lambda, MaxPooling2D, ReLU, Dropout, Concatenate, ZeroPadding2D, UpSampling2D, Activation
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras import regularizers
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, LambdaCallback, ReduceLROnPlateau, CSVLogger, TensorBoard
from tensorflow.keras import backend as K
import sklearn
import tensorflow_addons as tfa
%matplotlib inline

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('INFO')
import logging
tf.get_logger().setLevel(logging.ERROR)

In [5]:
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
from tensorflow.python.keras import backend as K
import tensorflow.keras as keras

In [6]:
# categories_1 = {
#     'flat': [0,1],
#     'construction': [2,3,4],
#     'object': [5,6,7],
#     'nature': [8,9],
#     'human': [11,12],
#     'vehicle':[13,14,15,16,17,18]
# }

In [7]:
hyperparameters={
    'cls': 'human',
    'targets': ['image_id'],
    'height': 128, # 512 
    'width': 256 , # 1024
    'channels': 3, 
    'batch_size': 8, 
    'epochs': 1, 
    'num_tasks': 7, ### MODIFY
    'initializer': 'he_uniform', 
    'reg_lambda': 1e-3, 
    'output': [2]*7, ### MODIFY === [2]*number of tasks
    'attrPerTask': 2,
    'lr': 1e-4, ### MODIFY
    'is_trained': False, 
    'dropout_prob': 0.3,
    'enable_cs': False, 
    'enable_sluice': False,
    'initial_percent':0.1,
    'initial_train_epoch': 7, ### NEED TO SEE----- Done
    'increment_train_epoch': 7, ### NEED TO SEE --- Done
    'uncertainity_repeat': 5, 
    'num_uncertain_elements': 148, 
    'additional_epoch': 5, ### NEED TO SEE
    'pretraining_epochs': 3, ### -- make it 2 or originally 1 - Done
    'train_initial_batches': 74, #6103, ### NEED TO SEE ------------- Done
    'enable_additional': False,
    'additional_attr_count':2,
    'all_updates':False,
    'initial_percent_val': 0.9, ### Added Now
    'split_index': 0 ### Added Now
    }

In [8]:
def preprocess(hyperparameters):
    
    train_files = pd.DataFrame(glob.glob("../input/cityscapes-train/*/*/*.png"), columns=['idx'])
    val_files = pd.DataFrame(glob.glob("../input/cityscapes/val/*.png"), columns=['idx'])
    
    train = train_files[:(len(train_files)//hyperparameters['batch_size'])*hyperparameters['batch_size']]
    val = val_files[:(len(val_files)//hyperparameters['batch_size'])*hyperparameters['batch_size']]
    
    return (train, val)

In [9]:
(train, val) = preprocess(hyperparameters)

In [10]:
# Parameters
params = {'dim': (hyperparameters['height'],hyperparameters['width']),
          'batch_size': hyperparameters['batch_size'],
          'n_channels': hyperparameters['channels'],
          'shuffle': True}

# Datasets
partition = {}
partition['train'] = train.idx.values
partition['val'] =  val.idx.values

tasks=['flat', 'construction', 'object', 'sky', 'nature', 'human', 'vehicle']

labels = {}
for i in partition['train']:
    k = i.split("/")
    p=[]
    for t in tasks:
        p.append("/".join(k[0:2]+['cityscapestasklabels/cityscapes/'+t+'/'+'train'] + [k[-1]]))
    labels[i]=p
    
for i in partition['val']:
    k = i.split("/")
    p=[]
    for t in tasks:
        p.append("/".join(k[0:2]+['cityscapestasklabels/cityscapes/'+t+'/'+'val'] + [k[-1]]))
    labels[i]=p

In [11]:
def augment_using_layers(images, masks, size=None, val=False):
    
    h_s = size[0]
    w_s = size[1]
    
    def aug(height=h_s, width=w_s, validation=val):
        
        resize = tf.keras.layers.Resizing(height, width, interpolation='nearest')
        
        flip = tf.keras.layers.RandomFlip(mode="horizontal")
        
        rota = tf.keras.layers.RandomRotation(0.1, fill_mode='wrap', interpolation='nearest')
        
        trans = tf.keras.layers.RandomTranslation(height_factor=(-0.1, 0.1),
                                            width_factor=(-0.1, 0.1), 
                                            fill_mode='wrap',interpolation='nearest')
        
        if validation:
            layers = [resize] 
        else:
            layers = [resize, flip, rota]
        aug_model = tf.keras.Sequential(layers)

        return aug_model
    
    aug = aug()
    
    if masks is not None:
        mask = tf.stack(masks, -1)
        mask = tf.cast(mask, 'float32')
        images_mask = tf.concat([images, mask], -1)   
    else:
        images_mask = tf.cast(images, 'float32')
    
    images_mask = aug(images_mask)  
    
    if masks is not None:
        image = images_mask[:,:,0:3]
        labels=[]
        for i in range(hyperparameters['num_tasks']):
            labels.append(images_mask[:, :, 3+i])
        return image/255.0, labels
    else:
        image = images_mask[:,:,0:3]
        mask = None
        return image/255.0, None

In [12]:
class DataGenerator(keras.utils.Sequence):

    def __init__(self, list_IDs, labels, batch_size=32, dim=(512,512), n_channels=3,shuffle=True, is_validation=False):
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.is_validation = is_validation
        self.on_epoch_end()

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y1 = np.empty((self.batch_size, *self.dim), dtype=int)
        y2 = np.empty((self.batch_size, *self.dim), dtype=int)
        y3 = np.empty((self.batch_size, *self.dim), dtype=int)
        y4 = np.empty((self.batch_size, *self.dim), dtype=int)
        y5 = np.empty((self.batch_size, *self.dim), dtype=int)
        y6 = np.empty((self.batch_size, *self.dim), dtype=int)
        y7 = np.empty((self.batch_size, *self.dim), dtype=int)
        
        # print(list_IDs_temp)
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            if type(ID) == list or type(ID) == np.ndarray:
                k = ID[0]
            else:
                k=ID
                
            #print(i, ID)
            #print(self.labels[k])
            #print(" ")
            #print()
            
            # read image
            img = cv2.imread(k)

            # read labels        
            img1 = cv2.imread(self.labels[k][0])[:,:,0]
            img2 = cv2.imread(self.labels[k][1])[:,:,0]
            img3 = cv2.imread(self.labels[k][2])[:,:,0]
            img4 = cv2.imread(self.labels[k][3])[:,:,0]
            img5 = cv2.imread(self.labels[k][4])[:,:,0]
            img6 = cv2.imread(self.labels[k][5])[:,:,0]
            img7 = cv2.imread(self.labels[k][6])[:,:,0]

            labs=[img1,img2,img3,img4,img5,img6,img7]

            # aug. dataset
            img_1, labs_1 = augment_using_layers(img, labs, (hyperparameters['height'], hyperparameters['width']), self.is_validation)
            
            X[i,] = img_1
            
            y1[i, ] = labs_1[0]
            y2[i, ] = labs_1[1]
            y3[i, ] = labs_1[2]
            y4[i, ] = labs_1[3]
            y5[i, ] = labs_1[4]
            y6[i, ] = labs_1[5]
            y7[i, ] = labs_1[6]

        return X, [y1, y2, y3, y4, y5, y6, y7]

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, Y = self.__data_generation(list_IDs_temp)

        return X, Y

In [13]:
class MultipleGenerator(keras.utils.Sequence):
    
    def __init__(self, list_IDs_1, list_IDs_2, labels, batch_size=32, dim=(512,512), n_channels=3,shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs_1 = list_IDs_1
        self.list_IDs_2 = list_IDs_2
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def on_epoch_end(self):
        self.indexes_1 = np.arange(len(self.list_IDs_1))
        self.indexes_2 = np.arange(len(self.list_IDs_2))

        if self.shuffle == True:
            np.random.shuffle(self.indexes_1)
            np.random.shuffle(self.indexes_2)
    
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs_1) / self.batch_size))

    def __data_generation(self, list_IDs_temp_1, list_IDs_temp_2):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X1 = np.empty((self.batch_size, *self.dim, self.n_channels))
        X2 = np.empty((self.batch_size, *self.dim, self.n_channels))
        y1 = np.empty((self.batch_size, *self.dim), dtype=int)
        y2 = np.empty((self.batch_size, *self.dim), dtype=int)
        y3 = np.empty((self.batch_size, *self.dim), dtype=int)
        y4 = np.empty((self.batch_size, *self.dim), dtype=int)
        y5 = np.empty((self.batch_size, *self.dim), dtype=int)
        y6 = np.empty((self.batch_size, *self.dim), dtype=int)
        y7 = np.empty((self.batch_size, *self.dim), dtype=int)       
        
        # Generate data
        for i, ID in enumerate(list_IDs_temp_1):      
            
            # Store sample
            img = cv2.imread(ID[0])
            
            img1 = cv2.imread(self.labels[ID[0]][0])[:,:,0]
            img2 = cv2.imread(self.labels[ID[0]][1])[:,:,0]
            img3 = cv2.imread(self.labels[ID[0]][2])[:,:,0]
            img4 = cv2.imread(self.labels[ID[0]][3])[:,:,0]
            img5 = cv2.imread(self.labels[ID[0]][4])[:,:,0]
            img6 = cv2.imread(self.labels[ID[0]][5])[:,:,0]
            img7 = cv2.imread(self.labels[ID[0]][6])[:,:,0]
            
            labs=[img1,img2,img3,img4,img5,img6,img7]
            
            # aug. dataset
            img_1, labs_1 = augment_using_layers(img, labs, (hyperparameters['height'], hyperparameters['width']))            
            X1[i,] = img_1
                        
            y1[i, ] = labs_1[0]
            y2[i, ] = labs_1[1]
            y3[i, ] = labs_1[2]
            y4[i, ] = labs_1[3]
            y5[i, ] = labs_1[4]
            y6[i, ] = labs_1[5]
            y7[i, ] = labs_1[6]
            
        for i, ID in enumerate(list_IDs_temp_2):
            
            img = cv2.imread(ID[0])
            img_1, labs_1 = augment_using_layers(img, None, (hyperparameters['height'], hyperparameters['width']))            
            
            X2[i,] = img_1

        return X1, X2, [y1, y2, y3, y4, y5, y6, y7]

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes_1 = self.indexes_1[index*self.batch_size:(index+1)*self.batch_size]
        indexes_2 = self.indexes_2[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp_1 = [self.list_IDs_1[k] for k in indexes_1]
        list_IDs_temp_2 = [self.list_IDs_2[k] for k in indexes_2]

        # Generate data
        X1, X2, Y = self.__data_generation(list_IDs_temp_1, list_IDs_temp_2)

        return X1, X2, Y

In [14]:
# from tf.keras.callbacks import LambdaCallback
# [LambdaCallback(on_epoch_end=generator.on_epoch_end)]

In [15]:
(train, val) = preprocess(hyperparameters)

# Generators
train_gen_full = DataGenerator(partition['train'], 
                                   labels, 
                                   dim = (hyperparameters['height'],hyperparameters['width']),
                                   batch_size= hyperparameters['batch_size'],
                                   n_channels = hyperparameters['channels'],
                                   shuffle = True)

val_gen = DataGenerator(partition['val'], 
                                   labels, 
                                   dim = (hyperparameters['height'],hyperparameters['width']),
                                   batch_size= hyperparameters['batch_size'],
                                   n_channels =hyperparameters['channels'],
                                   shuffle = False,
                                   is_validation=True)

pretrain_gen = MultipleGenerator(train.values, 
                                      val.values, 
                                      labels,
                                      batch_size= hyperparameters['batch_size'],
                                      dim = (hyperparameters['height'],hyperparameters['width']),
                                      n_channels = hyperparameters['channels'],
                                      shuffle = True)

In [16]:
# for i in labs_1:
#     print((i.numpy()==0).sum() + (i.numpy()==1).sum())
#     if ((i.numpy()==0).sum() + (i.numpy()==1).sum()) !=32768:
#         print(1)

## Architecture

In [17]:
def discriminator():
    
    X = Input((720,), name='input_disc')
    
    x = Dense(units = 512, activation = None, name='dense_1_disc')(X)
    x = BatchNormalization(name = 'bn_1_disc')(x)
    x = ReLU(name='relu_1_disc')(x)
    x = Dense(units = 512, activation = None, name='dense_2_disc')(x)
    x = BatchNormalization(name = 'bn_2_disc')(x)
    x = ReLU(name='relu_2_disc')(x)
    x = Dense(units = 1, activation = 'sigmoid', name='output_disc')(x)
    
    return X,x

In [18]:
def variationalAutoEncoder(embed_size=360):
    
    img_input = Input(shape=(hyperparameters['height'],hyperparameters['width'],hyperparameters['channels']), name = 'input_vae')

    x = Conv2D(64, (3, 3), activation='relu', padding='same',
               name='conv_1_vae')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same',
               name='conv_2_vae')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pooling_1_vae')(x)
    
    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same',
               name='conv_3_vae')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same',
               name='conv_4_vae')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pooling_2_vae')(x)
    
    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same',
               name='conv_5_vae')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same',
               name='conv_6_vae')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same',
               name='conv_7_vae')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pooling_3_vae')(x)
    
    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='conv_8_vae')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='conv_9_vae')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='conv_10_vae')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pooling_4_vae')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='conv_11_vae')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='conv_12_vae')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='conv_13_vae')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pooling_5_vae')(x)
    f5 = x
    
    x = Conv2D(32, (1, 1), activation='relu', padding='same',
               name='conv_14_vae')(x)    
    
    x = Flatten(name='flatten_1_vae')(x)
    mu = Dense(units = embed_size, activation = None, name='dense_1_vae')(x)
    
    x = Dense(units = 1024, activation = None, name='dense_2_vae')(mu)
    x = Reshape((4,8,32), name='reshape_vae')(x)
    x = Conv2DTranspose(512, kernel_size=(1,1), padding='same', use_bias=False, name='conv_15_vae')(x)
    
    x = (ZeroPadding2D((1, 1), name='zero_padding_1_vae'))(x)
    x = (Conv2D(512, (3, 3), padding='valid', name='conv_16_vae'))(x)
    x = (BatchNormalization(name = 'bn_1_vae'))(x)
    x = tf.keras.layers.ReLU(name='relu_vae_1')(x)
    
    x = (UpSampling2D((2, 2), name='upsampling_1_vae'))(x)
    x = (ZeroPadding2D((1, 1), name='zero_padding_2_vae'))(x)
    x = (Conv2D(256, (3, 3), padding='valid' ,name='conv_17_vae'))(x)
    x = (BatchNormalization(name = 'bn_2_vae'))(x)
    x = tf.keras.layers.ReLU(name='relu_vae_2')(x)

    x = (UpSampling2D((2, 2),  name='upsampling_2_vae'))(x)
    x = (ZeroPadding2D((1, 1),  name='zero_padding_3_vae'))(x)
    x = (Conv2D(128, (3, 3), padding='valid', name='conv_18_vae'))(x)
    x = (BatchNormalization(name = 'bn_3_vae'))(x)
    x = tf.keras.layers.ReLU(name='relu_vae_3')(x)

    x = (UpSampling2D((2, 2),  name='upsampling_3_vae'))(x)
    x = (ZeroPadding2D((1, 1),  name='zero_padding_4_vae'))(x)
    x = (Conv2D(128, (3, 3), padding='valid', name='conv_19_vae'))(x)
    x = (BatchNormalization(name = 'bn_4_vae'))(x)
    x = tf.keras.layers.ReLU(name='relu_vae_4')(x)

    x = (UpSampling2D((2, 2),  name='upsampling_4_vae'))(x)
    x = (ZeroPadding2D((1, 1),  name='zero_padding_5_vae'))(x)
    x = (Conv2D(3, (3, 3), padding='valid', name='conv_20_vae'))(x)
    x = (BatchNormalization(name = 'bn_5_vae'))(x)
    x = tf.keras.layers.ReLU(name='relu_vae_5')(x)

    x = (UpSampling2D((2, 2),  name='upsampling_5_vae'))(x)
    x = (ZeroPadding2D((1, 1),  name='zero_padding_6_vae'))(x)
    x = (Conv2D(3, (3, 3), padding='valid', name='conv_21_vae'))(x)
    x = (BatchNormalization(name = 'bn_6_vae'))(x)
    x = tf.keras.layers.ReLU(name='relu_vae_6')(x)

    return img_input, x, mu

In [19]:
# # defining my generator
# X, z, mu = variationalAutoEncoder()
# vae = Model(inputs = X, outputs = [z,mu])
# vae.compile(optimizer = keras.optimizers.RMSprop(learning_rate=hyperparameters['lr'],
#                                                  clipnorm=1.0 ))

In [20]:
def get_vgg_encoder(input_height=512, input_width=1024, pretrained='imagenet', channels=3):

    assert input_height % 32 == 0
    assert input_width % 32 == 0

    img_input = Input(shape=(input_height, input_width, channels), name = 'input_pred')

    x = Conv2D(64, (3, 3), activation='relu', padding='same',
               name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same',
               name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
    
    f1 = x
    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same',
               name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same',
               name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
    f2 = x

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same',
               name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same',
               name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same',
               name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
    f3 = x

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
    f4 = x

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    f5 = x

    return img_input, [f1, f2, f3, f4, f5]

In [21]:
def segnet_decoder(f, n_classes, n_up=3):

    assert n_up >= 2
    
    ## added ---------------------------------------------------------------
    f = Conv2D(360, (1, 1), padding='valid', name='conv_1_pred')(f) 
    f = GlobalAveragePooling2D(name = "gap_1_pred")(f)
    
    f = Dense(units = 360, activation = None, name='dense_1_pred')(f)  
    g = f
    f = Dense(units = 64*128*1, activation = None, name='dense_2_pred')(f)  
    f = Reshape((64,128,1), name='reshape_1_pred')(f)
    f = (Conv2D(256, (1, 1), padding='valid', name='conv_2_pred'))(f)
    ## --------------------------------------------------------------------

    o = f
    o = (ZeroPadding2D((1, 1), name = 'zero_1_pred'))(o)
    o = (Conv2D(512, (3, 3), padding='valid', name='conv_3_pred'))(o)
    o = (BatchNormalization(name = 'bn_1_pred'))(o)
    
    o = tf.keras.layers.SpatialDropout2D(0.3)(o)

    o = (UpSampling2D((2, 2), name = 'upsampling_1_pred'))(o)
    o = (ZeroPadding2D((1, 1), name = 'zero_2_pred'))(o)
    o = (Conv2D(256, (3, 3), padding='valid', name='conv_4_pred'))(o)
    o = (BatchNormalization(name = 'bn_2_pred'))(o)
    
#     o = (UpSampling2D((2, 2), name = 'upsampling_2_pred'))(o)
#     o = (ZeroPadding2D((1, 1), name = 'zero_3_pred'))(o)
#     o = (Conv2D(128, (3, 3), padding='valid', name = 'conv_5_pred'))(o)
#     o = (BatchNormalization(name = 'bn_3_pred'))(o)

#     o = (UpSampling2D((2, 2), name = 'upsampling_3_pred'))(o)
#     o = (ZeroPadding2D((1, 1), name = 'zero_4_pred'))(o)
#     o = (Conv2D(64, (3, 3), padding='valid', name = 'conv_6_pred'))(o)
#     o = (BatchNormalization(name = 'bn_4_pred'))(o)
    
    t1 = []
    for i in range(hyperparameters['num_tasks']):
        t1.append(Conv2D(1, (3, 3), padding='same', name = 'conv_7_'+ str(i) +'_pred', activation='sigmoid')(o))
    
    return t1, g

In [22]:
def get_segmentation_model(input, output):

    img_input = input
    o1,o2 = output

#     o_shape = Model(img_input, [o1,o2]).output_shape
#     i_shape = Model(img_input, [o1,o2]).input_shape
    
#     IMAGE_ORDERING = 'channels_last'
#     output_height = o_shape[0][1]
#     output_width = o_shape[0][2]
#     input_height = i_shape[1]
#     input_width = i_shape[2]
#     n_classes = o_shape[0][3]
    
    # o1 = (Reshape((output_height*output_width, -1), name='reshape_2_pred'))(o1)

    # o1 = (tf.keras.layers.Softmax(name='activation_1_pred'))(o1)
    model = Model(img_input, [o1, o2])
    
    # model.output_width = output_width
    # model.output_height = output_height
    
#     model.n_classes = n_classes
#     model.input_height = input_height
#     model.input_width = input_width
#     model.model_name = ""

    return model

def vgg_segnet(n_classes, input_height=416, input_width=608, encoder_level=3, channels=3):

    model = _segnet(n_classes, get_vgg_encoder,  input_height=input_height,
                    input_width=input_width, encoder_level=encoder_level, channels=channels)
    model.model_name = "vgg_segnet"
    return model

def _segnet(n_classes, encoder,  input_height=512, input_width=512,
            encoder_level=3, channels=3):

    img_input, levels = encoder(
        input_height=input_height,  input_width=input_width, channels=channels)

    feat = levels[encoder_level]
    o = segnet_decoder(feat, n_classes, n_up=3)
    model = get_segmentation_model(img_input, o)

    return model

In [23]:
# pred_model = vgg_segnet(n_classes = hyperparameters['num_tasks'], input_height=hyperparameters['height'], input_width=hyperparameters['width'], encoder_level=2, channels=hyperparameters['channels'])
# pred_model.compile(optimizer = keras.optimizers.SGD(learning_rate=hyperparameters['lr'],
#                                                 clipnorm=1.0 ))

## Active Learning

In [24]:
class ActiveLearning(keras.Model):
    def __init__(self, discriminator, generator, predictor, hyperparameters, trackers):
        super(ActiveLearning, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.predictor = predictor
        self.hyperparameters = hyperparameters
        self.trackers = trackers

    def compile(self, d_optimizer, g_optimizer, p_optimizer):
        super(ActiveLearning, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.p_optimizer = p_optimizer
                
    def train_step(self, real_images):
        
        # get labelled_x, unlabelled_x and labelled_y
        
        x = real_images
        labelled_x = x[0]
        unlabelled_x = x[1]
        labelled_y = x[2]

        ##### TRAIN THE PREDICTOR #####
        
        with tf.GradientTape() as tape:
            labelled_prediction_y,_ = self.predictor(labelled_x, training=True)

            re_1=[]
            for i in range(7):
                re_1.append(tf.reshape(labelled_y[i], (hyperparameters['batch_size'], -1)))

            re_2 = []
            for i in range(7):
                re_2.append(tf.reshape(labelled_prediction_y[i], (hyperparameters['batch_size'], -1)))

            weights = [{1: 1.0123, 0: 0.9879},
             {0: 0.6377, 1: 2.3142},
             {0: 0.5089, 1: 28.4156},
             {0: 0.5185, 1: 14.0129},
             {0: 0.5894, 1: 3.2964},
             {0: 0.5060, 1: 41.6062},
             {0: 0.5394, 1: 6.8425}]

            w = []
            for i in range(7):
                wts = np.zeros((re_1[i].shape))
                for j in range(re_1[i].shape[0]):
                    t = re_1[i][j].numpy()
                    t = t.astype('float64')
                    wts[j][t==0] = weights[i][0]
                    wts[j][t==1] = weights[i][1]

                wts=tf.convert_to_tensor(wts)
                predictor_loss = weighted_cross_entropy(re_1[i], re_2[i], sample_weights=wts)
                w.append(predictor_loss)
            w = tf.convert_to_tensor(w)
            w = tf.math.reduce_mean(w, axis=1) +  tf.cast(dice_coef_loss(re_1, re_2), tf.float32)

            #l = tfa.losses.SigmoidFocalCrossEntropy()
            # predictor_loss = l(re_1, re_2)
            #w = tf.math.reduce_mean(predictor_loss)

        trainable_vars = self.predictor.trainable_variables
        gradients = tape.gradient(w, trainable_vars)

        # Update weights
        self.p_optimizer.apply_gradients(zip(gradients, trainable_vars))        
        
#         # Compute output and latents
#         weights = {'0': 0.555, 
#                    '1': 5.017}
#         with tf.GradientTape() as tape:
#             labelled_prediction_y, _ = self.predictor(labelled_x, training=True)
# #             re_1 = tf.reshape(labelled_y, (hyperparameters['batch_size'], -1))
# #             index = tf.where(tf.not_equal(re_1, tf.constant(255, dtype=tf.int64)))
# #             a1 = tf.gather_nd(re_1, index)
# #             re_2 = tf.reshape(labelled_prediction_y, (hyperparameters['batch_size'], -1, hyperparameters['num_tasks']))
# #             a2 = tf.gather_nd(re_2, index) ## -----> check
# #             a1 = tf.convert_to_tensor(tf.keras.utils.to_categorical(a1, num_classes=hyperparameters['num_tasks']))    
# #             predictor_loss = soft_dice_loss(a1, a2)
#             re_1 = tf.reshape(labelled_y, (hyperparameters['batch_size'], -1))
#             index = tf.where(tf.not_equal(re_1, tf.constant(255, dtype=tf.int64)))
#             a1 = tf.gather_nd(re_1, index)
#             re_2 = tf.reshape(labelled_prediction_y, (hyperparameters['batch_size'], -1, hyperparameters['num_tasks']))
#             a2 = tf.gather_nd(re_2, index)

#             wts = np.ones((a1.shape))
#             for i in range(a1.shape[0]):
#                 if a1[i] == 0:
#                     wts[i] = weights['0']
#                 elif a1[i] == 1:
#                     wts[i] = weights['1']
#             wts=tf.convert_to_tensor(wts)

#             l = tf.keras.losses.SparseCategoricalCrossentropy(reduction = tf.keras.losses.Reduction.NONE)
#             predictor_loss = l(a1, a2, sample_weight=wts) # ----> 1
#             predictor_loss = tf.math.reduce_mean(predictor_loss)
            
#             # l = tf.keras.losses.SparseCategoricalCrossentropy(reduction = 'auto')
#             # predictor_loss = l(a1, a2) # ----> 1        

        
#         # Compute gradients
#         trainable_vars = self.predictor.trainable_variables
#         gradients = tape.gradient(predictor_loss, trainable_vars)
        
#         # Update weights
#         self.p_optimizer.apply_gradients(zip(gradients, trainable_vars)) 
        
        # ------------------------------------------------------------------------------------------------
        
        ##### TRAIN THE GENERATOR #####
        
        # Create labels for VAE
        labelled_disc_true = np.ones((self.hyperparameters['batch_size'],1))
        unlabelled_disc_fake = np.ones((self.hyperparameters['batch_size'],1))
        
        # Compute VAE outputs
        with tf.GradientTape() as tape:
            # Compute generator o/p
            labelled_vae_y, labelled_vae_latent = self.generator(labelled_x)
            unlabelled_vae_y, unlabelled_vae_latent = self.generator(unlabelled_x)
            
            # Calculate loss for VAE
            labelled_vae_loss = keras.losses.mean_squared_error(labelled_x, labelled_vae_y) # ----> 2
            unlabelled_vae_loss = keras.losses.mean_squared_error(unlabelled_x, unlabelled_vae_y) # ----> 2
            
            vae_loss = labelled_vae_loss + unlabelled_vae_loss #+ (self.advisory_param*disc_loss)
        
        # Compute gradients
        trainable_vars = self.generator.trainable_variables
        gradients = tape.gradient(vae_loss, trainable_vars)
        
        # Update weights
        self.g_optimizer.apply_gradients(zip(gradients, trainable_vars))         
        
        # ------------------------------------------------------------------------------------------------
        
        ##### TRAIN THE DISCRIMINATOR #####
        
        # Create disc labels
        labelled_disc_true = np.ones((self.hyperparameters['batch_size'],1))
        unlabelled_disc_true = np.zeros((self.hyperparameters['batch_size'],1))
        
        # Compute VAE latents
        _, labelled_vae_latent = self.generator(labelled_x, training = False)
        _, unlabelled_vae_latent = self.generator(unlabelled_x, training = False)
        
        # Compute predictor latents
        _, labelled_predictor_latent = self.predictor(labelled_x, training=False)
        _, unlabelled_predictor_latent = self.predictor(unlabelled_x, training=False)
        
#         labelled_predictor_latent = math_ops.mean(ops.convert_to_tensor(labelled_predictor_latent), axis=0)
#         unlabelled_predictor_latent = math_ops.mean(ops.convert_to_tensor(unlabelled_predictor_latent), axis=0)
        
        # Join vae and predictor latents
        labelled_disc_in = tf.concat([labelled_vae_latent,labelled_predictor_latent],axis=1)
        unlabelled_disc_in = tf.concat([unlabelled_vae_latent,unlabelled_predictor_latent],axis=1)
        
        # Compute disc output
        with tf.GradientTape() as tape:
            labelled_disc_y = self.discriminator(labelled_disc_in,training=True)
            unlabelled_disc_y = self.discriminator(unlabelled_disc_in,training=True)
            
            labelled_disc_loss = keras.losses.binary_crossentropy(labelled_disc_true, labelled_disc_y) # ----> 3
            unlabelled_dic_loss = keras.losses.binary_crossentropy(unlabelled_disc_true, unlabelled_disc_y) # ----> 3
            
            disc_loss = labelled_disc_loss + unlabelled_dic_loss
        
        # Compute gradients
        trainable_vars = self.discriminator.trainable_variables
        gradients = tape.gradient(disc_loss, trainable_vars)
        
        # Update weights
        self.d_optimizer.apply_gradients(zip(gradients, trainable_vars)) 
    
        # ------------------------------------------------------------------------------------------------
        
        # Computing Metrics
        
        # For predictor
#         wts = np.ones((a1.shape))
#         for i in range(a1.shape[0]):
#             if a1[i] == 0:
#                 wts[i] = weights['0']
#             elif a1[i] == 1:
#                 wts[i] = weights['1']
#         wts=tf.convert_to_tensor(wts)
        
#         predictor_loss = l(a1, a2, sample_weight=wts) # ----> 1
#         predictor_loss = tf.math.reduce_mean(predictor_loss)#soft_dice_loss(a1, a2)
#         predictor_loss = l(re_1, re_2)
#         predictor_loss = tf.math.reduce_mean(predictor_loss)
        self.trackers['loss_tracker_predictor'].update_state(w) # predictor_loss
        
#         re_1 = tf.reshape(labelled_y, (self.hyperparameters['batch_size'], -1))
#         index = tf.where(tf.not_equal(re_1, tf.constant(255, dtype=tf.int64)))
#         a1 = tf.gather_nd(re_1, index)
#         re_2 = tf.reshape(labelled_prediction_y, (hyperparameters['batch_size'], -1, hyperparameters['num_tasks']))
#         a2 = tf.gather_nd(re_2, index) ## -----> check
        
        # self.trackers['acc_metric_predictor'].update_state(a1, a2)
        self.trackers['acc_metric_predictor'].update_state(re_1, re_2)
        
        # f1 for predictor

#         a2 = np.argmax(a2.numpy(), axis=1)
#         a1 = a1.numpy()
#         l1 = f1_score(a1, a2, average='weighted', zero_division=1)
#         l2 = f1_score(a1, a2, labels=list(range(0,self.hyperparameters['num_tasks'])), average=None, zero_division=1)
#         cm = confusion_matrix(a1,a2, labels=list(range(0,self.hyperparameters['num_tasks'])))
#         FP = cm.sum(axis=0) - np.diag(cm)  
#         FN = cm.sum(axis=1) - np.diag(cm)
#         TP = np.diag(cm)
#         TN = cm.sum() - (FP + FN + TP)
        
#         self.trackers['f1_predictor'].update_state(l1)
#         self.trackers['IOU_predictor'].update_state(a1,a2)
#         rec = sklearn.metrics.recall_score(a1,a2, average=None, labels=list(range(0,self.hyperparameters['num_tasks'])), zero_division=1)
        for task in range(self.hyperparameters['num_tasks']):
            # self.trackers['individual_f1_predictor'][task].update_state(l2[task])
            acc=tf.keras.metrics.BinaryAccuracy()
            accuracy = acc(re_1[task], re_2[task])
            #acc = rec[task]
            #self.trackers['individual_acc_predictor'][task].update_state(acc)
            self.trackers['individual_acc_predictor'][task].update_state(accuracy)
            self.trackers['individual_IOU_predictor'][task].update_state(re_1[task], tf.cast(re_2[task]>=0.5, tf.int64))
        
        # For VAE
        self.trackers['loss_tracker_generator'].update_state(labelled_x, labelled_vae_y)
        self.trackers['loss_tracker_generator'].update_state(unlabelled_x, unlabelled_vae_y)
        # For Discriminator
        self.trackers['loss_tracker_disc'].update_state(labelled_disc_true,labelled_disc_y)
        self.trackers['loss_tracker_disc'].update_state(unlabelled_disc_true,unlabelled_disc_y)
        self.trackers['acc_tracker_disc'].update_state(labelled_disc_true,labelled_disc_y)
        self.trackers['acc_tracker_disc'].update_state(unlabelled_disc_true,unlabelled_disc_y)
            
        ret_dic = {"loss_predictor_total": self.trackers['loss_tracker_predictor'].result(), # loss_tracker_predictor.result(), 
                   "acc_predictor":self.trackers['acc_metric_predictor'].result(), # acc_metric_predictor.result(), 
                   #'f1_predictor': self.trackers['f1_predictor'].result(),
                   "loss_VAE":  self.trackers['loss_tracker_generator'].result(), # loss_tracker_generator.result(),
                   "loss_disc": self.trackers['loss_tracker_disc'].result(), # loss_tracker_disc.result(),
                   "acc_disc": self.trackers['acc_tracker_disc'].result(), # acc_tracker_disc.result()
                   #"IOU_predictor": self.trackers['IOU_predictor'].result()}
                  }

        for i in range(hyperparameters['num_tasks']):
            ret_dic["acc_predictor_"+str(i)] = self.trackers['individual_acc_predictor'][i].result() # individual_acc_metric_predictor[i].result()
            ret_dic["IOU_predictor_"+str(i)] = self.trackers['individual_IOU_predictor'][i].result() # individual_acc_metric_predictor[i].result()
            #ret_dic["f1_predictor_"+str(i)] = self.trackers['individual_f1_predictor'][i].result() # individual_acc_metric_predictor[i].result()

        return ret_dic
    
    def call(self, x):
        return
    
    def test_step(self, real_images):
        
        x = real_images
        labelled_x = x[0]
        labelled_y = x[1]
        
        # Predictor step
        labelled_prediction_y, labelled_predictor_latent = self.predictor(labelled_x, training=False)
        
        # Generator step
        labelled_vae_y, labelled_vae_latent = self.generator(labelled_x, training=False)
        
        # Discriminator step
#         labelled_predictor_latent = math_ops.mean(ops.convert_to_tensor(labelled_predictor_latent), axis=0)
        labelled_disc_in = tf.concat([labelled_vae_latent,labelled_predictor_latent],axis=1)
        
        labelled_disc_y = self.discriminator(labelled_disc_in,training=False)
        
        # Updating metrics
        
        # For Predictor
#         re_1 = tf.reshape(labelled_y, (hyperparameters['batch_size'], -1))
#         index = tf.where(tf.not_equal(re_1, tf.constant(255, dtype=tf.int64)))
#         a1 = tf.gather_nd(re_1, index)
#         re_2 = tf.reshape(labelled_prediction_y, (hyperparameters['batch_size'], -1, hyperparameters['num_tasks']))
#         a2 = tf.gather_nd(re_2, index) ## -----> check
#         a1 = tf.convert_to_tensor(tf.keras.utils.to_categorical(a1, num_classes=hyperparameters['num_tasks']))    
#         predictor_loss = soft_dice_loss(a1, a2)

#         re_1 = tf.reshape(labelled_y, (hyperparameters['batch_size'], -1))
#         index = tf.where(tf.not_equal(re_1, tf.constant(255, dtype=tf.int64)))
#         a1 = tf.gather_nd(re_1, index)
#         re_2 = tf.reshape(labelled_prediction_y, (hyperparameters['batch_size'], -1, hyperparameters['num_tasks']))
#         a2 = tf.gather_nd(re_2, index)

#         l = tf.keras.losses.SparseCategoricalCrossentropy(reduction = 'auto')
#         predictor_loss = l(a1, a2) # ----> 1 
#         self.trackers['loss_tracker_predictor'].update_state(a1,a2)

#         re_1 = tf.reshape(labelled_y, (self.hyperparameters['batch_size'], -1))
#         index = tf.where(tf.not_equal(re_1, tf.constant(255, dtype=tf.int64)))
#         a1 = tf.gather_nd(re_1, index)
#         re_2 = tf.reshape(labelled_prediction_y, (hyperparameters['batch_size'], -1, hyperparameters['num_tasks']))
#         a2 = tf.gather_nd(re_2, index) ## -----> check
        
#         self.trackers['acc_metric_predictor'].update_state(a1, a2)

        re_1=[]
        for i in range(7):
            re_1.append(tf.reshape(labelled_y[i], (hyperparameters['batch_size'], -1)))

        re_2 = []
        for i in range(7):
            re_2.append(tf.reshape(labelled_prediction_y[i], (hyperparameters['batch_size'], -1)))
            
        w = []
        for i in range(7):
            wts = np.ones((re_1[i].shape))
            wts=tf.convert_to_tensor(wts)
            predictor_loss = weighted_cross_entropy(re_1[i], re_2[i], sample_weights=wts)
            w.append(predictor_loss)
        w = tf.convert_to_tensor(w)
        w = tf.math.reduce_mean(w, axis=1) +  tf.cast(dice_coef_loss(re_1, re_2), tf.float32)
        

#         l = tf.keras.losses.BinaryCrossentropy(from_logits=False, reduction = tf.keras.losses.Reduction.NONE)
#         predictor_loss = l(re_1, re_2) # ----> 
#         predictor_loss = tf.math.reduce_mean(predictor_loss)

#         l = tfa.losses.SigmoidFocalCrossEntropy()
#         predictor_loss = l(re_1, re_2)
#         w = tf.math.reduce_mean(predictor_loss)
#         w = dice_coef_loss(re_1, re_2)
        self.trackers['loss_tracker_predictor'].update_state(w)
        self.trackers['acc_metric_predictor'].update_state(re_1, re_2)
        
        # f1 for predictor
#         a2 = np.argmax(a2.numpy(), axis=1)
#         a1 = a1.numpy()
#         l1 = f1_score(a1, a2, average='weighted', zero_division=1)
#         l2 = f1_score(a1, a2, labels=list(range(0,self.hyperparameters['num_tasks'])), average=None, zero_division=1)
#         cm = confusion_matrix(a1,a2, labels=list(range(0,self.hyperparameters['num_tasks'])))
#         FP = cm.sum(axis=0) - np.diag(cm)  
#         FN = cm.sum(axis=1) - np.diag(cm)
#         TP = np.diag(cm)
#         TN = cm.sum() - (FP + FN + TP)
        
        #self.trackers['f1_predictor'].update_state(l1)
        #self.trackers['IOU_predictor'].update_state(a1,a2)
        #rec = sklearn.metrics.recall_score(a1,a2, average=None, labels=list(range(0,self.hyperparameters['num_tasks'])), zero_division=1)
        for task in range(self.hyperparameters['num_tasks']):
            #self.trackers['individual_f1_predictor'][task].update_state(l2[task])
            # acc = rec[task]
            # self.trackers['individual_acc_predictor'][task].update_state(acc)
            acc=tf.keras.metrics.BinaryAccuracy()
            accuracy = acc(re_1[task], re_2[task])
            #acc = rec[task]
            #self.trackers['individual_acc_predictor'][task].update_state(acc)
            self.trackers['individual_acc_predictor'][task].update_state(accuracy)
            self.trackers['individual_IOU_predictor'][task].update_state(re_1[task], tf.cast(re_2[task]>=0.5, tf.int64))

        self.trackers['loss_tracker_generator'].update_state(labelled_x, labelled_vae_y)
        
        # For Discriminator
        labelled_disc_true = np.ones((self.hyperparameters['batch_size'],1))
        self.trackers['loss_tracker_disc'].update_state(labelled_disc_true,labelled_disc_y)
        self.trackers['acc_tracker_disc'].update_state(labelled_disc_true,labelled_disc_y)
        
        ret_dic = {"loss_predictor_total": self.trackers['loss_tracker_predictor'].result(), # loss_tracker_predictor.result(), 
                   "acc_predictor":self.trackers['acc_metric_predictor'].result(), # acc_metric_predictor.result(), 
                   #'f1_predictor': self.trackers['f1_predictor'].result(),
                   "loss_VAE":  self.trackers['loss_tracker_generator'].result(), # loss_tracker_generator.result(),
                   "loss_disc": self.trackers['loss_tracker_disc'].result(), # loss_tracker_disc.result(),
                   "acc_disc": self.trackers['acc_tracker_disc'].result(), # acc_tracker_disc.result()
                   #"IOU_predictor": self.trackers['IOU_predictor'].result()}
                  }
        
        for i in range(hyperparameters['num_tasks']):
            ret_dic["acc_predictor_"+str(i)] = self.trackers['individual_acc_predictor'][i].result() # individual_acc_metric_predictor[i].result()
            #ret_dic["f1_predictor_"+str(i)] = self.trackers['individual_f1_predictor'][i].result() # individual_acc_metric_predictor[i].result()
            ret_dic["IOU_predictor_"+str(i)] = self.trackers['individual_IOU_predictor'][i].result() # individual_acc_metric_predictor[i].result()
       
        return ret_dic        
    
    def predict_step(self, real_images):
        unlabelled_x, unlabelled_y = real_images
        
        # Predictor step
        unlabelled_prediction_y, unlabelled_predictor_latent = self.predictor(unlabelled_x, training=False)
        
        # Generator step
        unlabelled_vae_y, unlabelled_vae_latent = self.generator(unlabelled_x, training=False)
        
        # Discriminator step
#         unlabelled_predictor_latent = math_ops.mean(ops.convert_to_tensor(unlabelled_predictor_latent), axis=0)
        unlabelled_disc_in = tf.concat([unlabelled_vae_latent,unlabelled_predictor_latent],axis=1)
        
        unlabelled_disc_y = self.discriminator(unlabelled_disc_in,training=False)
        
        return unlabelled_prediction_y #, unlabelled_disc_y#, unlabelled_y

    @property
    def metrics(self):
        # We list our `Metric` objects here so that `reset_states()` can be
        # called automatically at the start of each epoch
        # or at the start of `evaluate()`.
        # If you don't implement this property, you have to call
        # `reset_states()` yourself at the time of your choosing.
        #return [self.trackers["loss_tracker_predictor"], self.trackers["acc_metric_predictor"], self.trackers['f1_predictor'], self.trackers["loss_tracker_generator"], self.trackers["loss_tracker_disc"], self.trackers["acc_tracker_disc"], self.trackers["f1_predictor"], self.trackers["IOU_predictor"]] + self.trackers["individual_f1_predictor"] + self.trackers["individual_acc_predictor"]
        return [self.trackers["loss_tracker_predictor"], self.trackers["acc_metric_predictor"], self.trackers["loss_tracker_generator"], self.trackers["loss_tracker_disc"], self.trackers["acc_tracker_disc"]] + self.trackers["individual_acc_predictor"] + self.trackers['individual_IOU_predictor']

In [25]:
def weighted_cross_entropy(y_true, y_pred, sample_weights):
    y_true = tf.cast(y_true,y_pred.dtype)
    q1 = -1*(tf.math.multiply(y_true,tf.math.log(y_pred)) + tf.math.multiply((1.0-y_true),tf.math.log(1.0-y_pred)))
    sample_weights = tf.cast(sample_weights,q1.dtype)
    z = tf.math.multiply(q1,sample_weights)
    
    return tf.math.reduce_mean(z,axis=1)

def dice_coef(y_true, y_pred):
    smooth = 1.0
    y_true_f = tf.cast(K.flatten(y_true), tf.float64)
    y_pred_f = tf.cast(K.flatten(y_pred), tf.float64)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    t = []
    for i in range(7):
        t.append(1 - dice_coef(y_true[i], y_pred[i]))
        
    return tf.convert_to_tensor(t)

In [26]:
def uncertainity(probs, weights):
    lis = []
    lis_output = []
    for i in range(hyperparameters['num_tasks']):
        attr_output = probs[i]
        w = weights[:,i]
        k = -1* np.sum(attr_output*np.log(attr_output),axis=1)
        lis_output.append(k)
        lis.append(w*k)
    
    variance = np.var(np.array(lis),axis=0)
    return np.array(lis).sum(axis=0), variance

def getIndices(output, hyperparameters ,pretrain=False):
    if pretrain == True:
        count =  hyperparameters['train_initial_batches']*hyperparameters['batch_size']
        if ((output<=0.5).sum())>=count:
            sort = np.argwhere(output<=0.5)[:,0]
            return sort
        else:
            selection = (int((hyperparameters['train_initial_batches']*hyperparameters['batch_size'])/1000)+1)*1000
            sort = np.argpartition((output)[:,0], selection)
            return sort[:selection]
    else:
        count = hyperparameters['num_uncertain_elements']
        if ((output<=0.5).sum())>=count:
            sort = np.argwhere(output<=0.5)[:,0]
            return sort
        else:
            selection = (int(hyperparameters['num_uncertain_elements']/1000)+1)*1000
            sort = np.argpartition((output)[:,0], selection)
            return sort[:selection]
        
def divide_data(train, initial = False):
    num_samples = train.values.shape[0]
    
    if initial:
        idx = random.sample(list(np.arange(num_samples)), ((int(hyperparameters['initial_percent_val']*num_samples)//hyperparameters['batch_size'])*hyperparameters['batch_size']))
    else:
        idx = random.sample(list(np.arange(num_samples)), ((int(hyperparameters['initial_percent']*num_samples)//hyperparameters['batch_size'])*hyperparameters['batch_size']))

    return pd.DataFrame(train.values[idx,:], columns=train.columns), idx

In [27]:
# def dice_coef_loss(y_true, y_pred):
#     intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
#     return 1-((2. * intersection + 1) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + 1))

def soft_dice_loss(y_true, y_pred, epsilon=1e-6): 
    ''' 
    Soft dice loss calculation for arbitrary batch size, number of classes, and number of spatial dimensions.
    Assumes the `channels_last` format.
  
    # Arguments
        y_true: b x X x Y( x Z...) x c One hot encoding of ground truth
        y_pred: b x X x Y( x Z...) x c Network output, must sum to 1 over c channel (such as after softmax) 
        epsilon: Used for numerical stability to avoid divide by zero errors
    
    # References
        V-Net: Fully Convolutional Neural Networks for Volumetric Medical Image Segmentation 
        https://arxiv.org/abs/1606.04797
        More details on Dice loss formulation 
        https://mediatum.ub.tum.de/doc/1395260/1395260.pdf (page 72)
        
        Adapted from https://github.com/Lasagne/Recipes/issues/99#issuecomment-347775022
    '''
    
    # print(y_true.shape, y_pred.shape)
    # skip the batch and class axis for calculating Dice score
    axes = tuple(range(1, len(y_pred.shape)-1)) 
    # numerator = 2. * np.sum(y_pred * y_true, axes)
    numerator = 2. * tf.math.reduce_sum(y_pred * y_true, axes)
    # denominator = np.sum(np.square(y_pred) + np.square(y_true), axes)
    denominator = tf.math.reduce_sum(tf.add(tf.square(y_pred),tf.square(y_true)), axes)

    # return 1 - np.mean((numerator + epsilon) / (denominator + epsilon)) # average over classes and batch
    return 1 - tf.math.reduce_mean(tf.add(numerator,epsilon) / tf.add(denominator,epsilon)) # average over classes and batch

    # thanks @mfernezir for catching a bug in an earlier version of this implementation!

In [28]:
class CalculatingPredictions(tf.keras.callbacks.Callback):
    def __init__(self, filename, preds, test_gen, lr, is_validation=False):
        
        self.filename = filename
        self.preds = preds
        self.test_gen = test_gen
        self.lr = lr
        self.is_validation=is_validation

    def on_epoch_end(self, epoch, logs=None):
        
#         predict=self.model.evaluate(self.test_gen)
#         print(predict)
#         self.preds.append(predict)
#         k = np.array(self.preds)
#         if (self.is_validation==True):
#             np.save("./saved_history/" + self.filename + "_validation"+ str(epoch)+ ".npy", k)
#         else:
#             np.save("./saved_history/"+ self.filename + '_' +str(epoch)+ ".npy", k)
        
        date = datetime.datetime.now().strftime("%d - %b - %y - %H:%M:%S")
        
        if (self.is_validation==False and epoch%2==0):
            self.model.predictor.save_weights("./saved_history/models/pred_model_" + self.filename + "_epoch"+str(epoch)+'.h5')
            self.model.discriminator.save_weights("./saved_history/models/disc_model_" + self.filename +"_epoch"+str(epoch)+'.h5')
            self.model.generator.save_weights("./saved_history/models/vae_model_" + self.filename + "_epoch"+str(epoch)+'.h5')
        elif (self.is_validation==True):
            self.model.predictor.save_weights("./saved_history/models/val_pred_model_" + self.filename + "_epoch"+str(epoch)+'.h5')
            self.model.discriminator.save_weights("./saved_history/models/val_disc_model_" + self.filename +"_epoch"+str(epoch)+'.h5')
            self.model.generator.save_weights("./saved_history/models/val_vae_model_" + self.filename + "_epoch"+str(epoch)+'.h5')

In [29]:
!mkdir /kaggle/working/predictions
!mkdir /kaggle/working/saved_history/
!mkdir /kaggle/working/saved_history/models
!mkdir logs

In [30]:
break_point_ep = {'3': 1e-5,'6': 1e-5,'10': 1e-5}
splits = [0.1,0.15,0.2,0.25,0.3,0.35,0.4]

# defining metrics
trackers = {
#     'loss_tracker_predictor': tf.keras.metrics.Mean(name='loss_predictor_total'),
    "loss_tracker_predictor": tf.keras.metrics.Mean(name="loss_predictor_total"),# tf.keras.metrics.SparseCategoricalCrossentropy(name="loss_predictor_total"),
    "acc_metric_predictor": tf.keras.metrics.BinaryAccuracy (name="acc_predictor"),
    
    #'f1_predictor': tf.keras.metrics.Mean(name='f1_predictor'),
    #'IOU_predictor': tf.keras.metrics.MeanIoU(name='IOU_predictor', num_classes=hyperparameters['num_tasks']),

    "individual_acc_predictor": [tf.keras.metrics.Mean(name="acc_predictor_"+str(i)) for i in range(hyperparameters['num_tasks'])],
    "individual_IOU_predictor": [tf.keras.metrics.MeanIoU(name='IOU_predictor'+str(i), num_classes=2) for i in range(hyperparameters['num_tasks'])],
    
    #"individual_f1_predictor": [tf.keras.metrics.Mean(name="f1_predictor_"+str(i)) for i in range(hyperparameters['num_tasks'])],
    #"individual_f1_predictor": [tf.keras.metrics.Mean(name="f1_predictor_"+str(i)) for i in range(hyperparameters['num_tasks'])],
    "loss_tracker_generator": tf.keras.metrics.MeanSquaredError(name='loss_VAE'),
    "loss_tracker_disc":  tf.keras.metrics.BinaryCrossentropy(name='loss_disc'),
    "acc_tracker_disc": tf.keras.metrics.BinaryAccuracy("acc_disc")
}

In [31]:
## startTrain
def startTraining(filename, trackers, splits, break_point_ep,  validation_first = False, load_pred_model = False, load_vae = False, load_disc = False, further_training=False):
    tf.config.run_functions_eagerly(True)
    
    preds=[]
    validation_train_history=[] # new
    logdir = "./logs/" + filename
    filepath = "./saved_history/model/"+filename
    
    checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=False, mode='max', period = 1)
    csv_logger = CSVLogger('./saved_history/training_results_'+filename+'.csv', separator = ',', append=True)
    pre_train_logger = CSVLogger('./saved_history/pre_training_results_'+filename+'.csv', separator = ',', append=True)
    tensorboard_callback = TensorBoard(log_dir = logdir)
    pre_tensorboard_callback = TensorBoard(log_dir =" ./logs/pre_"+filename)
    
    # predictor
    pred_model = vgg_segnet(n_classes = hyperparameters['num_tasks'], input_height=hyperparameters['height'], input_width=hyperparameters['width'], encoder_level=2, channels=hyperparameters['channels'])
    pred_model.compile(optimizer = keras.optimizers.SGD(learning_rate=hyperparameters['lr'],
                                                    clipnorm=1.0 ))
    
    if load_pred_model:
        print("Predictor weights loading ...")
        # pred_model.load_weights("./saved_history/models/pred_model_from_scratch_vanilla_4.h5", by_name=True)
        
    # defining my discriminator
    disc_in, disc_out = discriminator()
    disc = Model(inputs = disc_in, outputs = disc_out)
    disc.compile(optimizer = keras.optimizers.SGD(learning_rate=hyperparameters['lr'],
                                                  clipnorm=1.0 ))

    if load_disc:
        print("Discriminator weights loading ...")
        # disc.load_weights()

    # defining my generator
    X, z, mu = variationalAutoEncoder()
    vae = Model(inputs = X, outputs = [z,mu])
    vae.compile(optimizer = keras.optimizers.RMSprop(learning_rate=hyperparameters['lr'],
                                                     clipnorm=1.0 ))
    if load_vae:
        print("VAE weights loading ...")
        # vae.load_weights()
        
    # Instantiate AL model
    AL_model = ActiveLearning(discriminator=disc, generator=vae, predictor=pred_model, hyperparameters=hyperparameters, trackers = trackers)
    AL_model.compile(
        d_optimizer=keras.optimizers.SGD(learning_rate=hyperparameters['lr'],clipnorm=1.0 ),
        g_optimizer= keras.optimizers.RMSprop(learning_rate=hyperparameters['lr'],clipnorm=1.0),
        p_optimizer=keras.optimizers.SGD(learning_rate=hyperparameters['lr'],clipnorm=1.0 ))
    
    print('model loaded')
    
    (train, val) = preprocess(hyperparameters)
    
    # Generators
    train_gen_full = DataGenerator(partition['train'], 
                                       labels, 
                                       dim = (hyperparameters['height'],hyperparameters['width']),
                                       batch_size= hyperparameters['batch_size'],
                                       n_channels = hyperparameters['channels'],
                                       shuffle = True)

    val_gen = DataGenerator(partition['val'], 
                                       labels, 
                                       dim = (hyperparameters['height'],hyperparameters['width']),
                                       batch_size= hyperparameters['batch_size'],
                                       n_channels =hyperparameters['channels'],
                                       shuffle = False, is_validation=True)
    
    if validation_first == True:
        labelled_pretrain, idx_prelabelled = divide_data(val, initial=True)
        idx_preunlabelled = list(np.setdiff1d(list(range(val.shape[0])), idx_prelabelled))
        unlabelled_pretrain = pd.DataFrame(val.values[idx_preunlabelled,:], columns=val.columns)
        
        pretrain_gen = MultipleGenerator(labelled_pretrain.values, 
                                              unlabelled_pretrain.values, 
                                              labels,
                                              batch_size= hyperparameters['batch_size'],
                                              dim = (hyperparameters['height'],hyperparameters['width']),
                                              n_channels = hyperparameters['channels'],
                                              shuffle=True)
        
        labelled_pretrain_gen = DataGenerator(unlabelled_pretrain.values, 
                                       labels, 
                                       dim = (hyperparameters['height'],hyperparameters['width']),
                                       batch_size= hyperparameters['batch_size'],
                                       n_channels = hyperparameters['channels'],
                                       shuffle = False)
        val_history = AL_model.fit(pretrain_gen, epochs = hyperparameters['pretraining_epochs'], steps_per_epoch = 55, validation_data=val_gen, callbacks = [CalculatingPredictions(filename, preds, val_gen, 0.01, True) , pre_tensorboard_callback, pre_train_logger], verbose=1)
        validation_train_history.append(val_history.history)
        with open("./saved_history/pretraining_history_list_"+filename+'.json', 'w') as f:
            json.dump(validation_train_history, f, indent=2)
            
        labelled_train, idx_labelled = divide_data(train)
        idx_unlabelled = list(np.setdiff1d(list(range(train.shape[0])), idx_labelled))
        unlabelled_train = pd.DataFrame(train.values[idx_unlabelled,:], columns=train.columns)     
        
        train_gen = MultipleGenerator(labelled_train.values, 
                                              unlabelled_train.values, 
                                              labels,
                                              batch_size= hyperparameters['batch_size'],
                                              dim = (hyperparameters['height'],hyperparameters['width']),
                                              n_channels = hyperparameters['channels'],
                                              shuffle=True) 

        unlabelled_gen = DataGenerator(unlabelled_train.values, 
                                           labels, 
                                           dim = (hyperparameters['height'],hyperparameters['width']),
                                           batch_size= hyperparameters['batch_size'],
                                           n_channels = hyperparameters['channels'],
                                           shuffle = False)
        
        labelled_gen = DataGenerator(labelled_train.values, 
                                           labels, 
                                           dim = (hyperparameters['height'],hyperparameters['width']),
                                           batch_size= hyperparameters['batch_size'],
                                           n_channels = hyperparameters['channels'],
                                           shuffle = False)
    elif further_training==True:
    
        idx_labelled = np.load("./saved_history/idx_labelled_4.npy")
        labelled_train = train.iloc[idx_labelled, :]
        idx_unlabelled = list(np.setdiff1d(list(range(train.shape[0])), idx_labelled))
        unlabelled_train = pd.DataFrame(train.values[idx_unlabelled,:], columns=train.columns)

        train_gen = MultipleGenerator(labelled_train.values, 
                                              unlabelled_train.values, 
                                              labels,
                                              batch_size= hyperparameters['batch_size'],
                                              dim = (hyperparameters['height'],hyperparameters['width']),
                                              n_channels = hyperparameters['channels'],
                                              shuffle=True) 

        unlabelled_gen = DataGenerator(unlabelled_train.values, 
                                           labels, 
                                           dim = (hyperparameters['height'],hyperparameters['width']),
                                           batch_size= hyperparameters['batch_size'],
                                           n_channels = hyperparameters['channels'],
                                           shuffle = False)
        
        labelled_gen = DataGenerator(labelled_train.values, 
                                           labels, 
                                           dim = (hyperparameters['height'],hyperparameters['width']),
                                           batch_size= hyperparameters['batch_size'],
                                           n_channels = hyperparameters['channels'],
                                           shuffle = False) 
    else:
        
        labelled_train, idx_labelled = divide_data(train)
        idx_unlabelled = list(np.setdiff1d(list(range(train.shape[0])), idx_labelled))
        unlabelled_train = pd.DataFrame(train.values[idx_unlabelled,:], columns=train.columns)        

        train_gen = MultipleGenerator(labelled_train.values, 
                                              unlabelled_train.values, 
                                              labels,
                                              batch_size= hyperparameters['batch_size'],
                                              dim = (hyperparameters['height'],hyperparameters['width']),
                                              n_channels = hyperparameters['channels'],
                                              shuffle=True) 

        unlabelled_gen = DataGenerator(unlabelled_train.values, 
                                           labels, 
                                           dim = (hyperparameters['height'],hyperparameters['width']),
                                           batch_size= hyperparameters['batch_size'],
                                           n_channels = hyperparameters['channels'],
                                           shuffle = False)
        
        labelled_gen = DataGenerator(labelled_train.values, 
                                           labels, 
                                           dim = (hyperparameters['height'],hyperparameters['width']),
                                           batch_size= hyperparameters['batch_size'],
                                           n_channels = hyperparameters['channels'],
                                           shuffle = False) 
    
    history_list=[]
    if further_training==True:
        num_batches = 10 #idx_labelled.shape[0]//hyperparameters['batch_size']
        tensorboard_callback = TensorBoard(log_dir = 'AL_model_from_scratch_non_cs_09 - Jan - 21 - 12:50:36')

        iteration =5
        epoch_num = 13
        
        history = AL_model.fit(train_gen,initial_epoch = epoch_num, epochs=epoch_num+7, steps_per_epoch = num_batches, validation_data=val_gen,callbacks = [CalculatingPredictions(filename, preds, val_gen, 0.01), csv_logger], verbose = 1)
        history_list.append(history.history)
        pred_model.save_weights("./saved_history/models/pred_model_from_scratch_vanilla_"+str(iteration)+'.h5')
        disc.save_weights("./saved_history/models/disc_model_from_scratch_vanilla_"+str(iteration)+'.h5')
        vae.save_weights("./saved_history/models/vae_model_from_scratch_vanilla_"+str(iteration)+'.h5')
        with open("./saved_history/history_list_only_"+str(iteration)+".json", 'w') as f:
            json.dump(history_list, f, indent=2)

        with open("./saved_history/preds_only_"+str(iteration)+".json", 'w') as f:
            json.dump(preds, f, indent=2)
    else:
        test_predictions=[]
        indices_list = []
        epoch_num = 0        
        num_batches = (train.shape[0]*splits[hyperparameters['split_index']])//hyperparameters['batch_size']
                
        for iteration in range(len(splits)): 
            print(iteration, splits[iteration], len(train_gen))
            
            if iteration==0:
                # Initial training ---- change
                history = AL_model.fit(train_gen, epochs=hyperparameters['initial_train_epoch'], steps_per_epoch = num_batches, validation_data=val_gen, callbacks = [CalculatingPredictions(filename, preds, val_gen, 0.01), csv_logger, tensorboard_callback], verbose = 1)
                history_list.append(history.history)
                epoch_num+=hyperparameters['initial_train_epoch']
            else:
                # predictor
                pred_model = vgg_segnet(n_classes = hyperparameters['num_tasks'], input_height=hyperparameters['height'], input_width=hyperparameters['width'], encoder_level=2, channels=hyperparameters['channels'])
                pred_model.compile(optimizer = keras.optimizers.SGD(learning_rate=hyperparameters['lr'],
                                                                clipnorm=1.0 ))

                # pred_model.load_weights("./saved_history/models/pred_model_from_scratch_vanilla_4.h5", by_name=True)

                # defining my discriminator
                disc_in, disc_out = discriminator()
                disc = Model(inputs = disc_in, outputs = disc_out)
                disc.compile(optimizer = keras.optimizers.SGD(learning_rate=hyperparameters['lr'],
                                                              clipnorm=1.0 ))

                # defining my generator
                X, z, mu = variationalAutoEncoder()
                vae = Model(inputs = X, outputs = [z,mu])
                vae.compile(optimizer = keras.optimizers.RMSprop(learning_rate=hyperparameters['lr'],
                                                                 clipnorm=1.0 ))
                
                if validation_first==True:
                    pred_model.load_weights("./saved_history/models/val_pred_model_" + filename + "_epoch2"+'.h5', by_name=True)
                    vae.load_weights("./saved_history/models/val_vae_model_" + filename + "_epoch2"+'.h5', by_name=True)
                    disc.load_weights("./saved_history/models/val_disc_model_" + filename + "_epoch2"+'.h5', by_name=True)
                
                # Instantiate AL model
                AL_model = ActiveLearning(discriminator=disc, generator=vae, predictor=pred_model, hyperparameters=hyperparameters, trackers = trackers)
                AL_model.compile(
                    d_optimizer=keras.optimizers.SGD(learning_rate=hyperparameters['lr'],clipnorm=1.0 ),
                    g_optimizer= keras.optimizers.RMSprop(learning_rate=hyperparameters['lr'],clipnorm=1.0),
                    p_optimizer=keras.optimizers.SGD(learning_rate=hyperparameters['lr'],clipnorm=1.0 ))

                print('model loaded')

                # Increment training --- change
                history = AL_model.fit(train_gen, initial_epoch = epoch_num, epochs=epoch_num+hyperparameters['increment_train_epoch'] + iteration, steps_per_epoch = num_batches, validation_data=val_gen,callbacks = [CalculatingPredictions(filename, preds, val_gen, 0.01), csv_logger, tensorboard_callback], verbose = 1)
                history_list.append(history.history)
                epoch_num+=hyperparameters['increment_train_epoch']+iteration
        
            pred_model.save_weights("./saved_history/models/pred_model_" + filename +"_iteration_"+str(iteration)+'.h5')
            disc.save_weights("./saved_history/models/disc_model_" + filename + "_iteration_"+str(iteration)+'.h5')
            vae.save_weights("./saved_history/models/vae_model_"+ filename + "_iteration_"+str(iteration)+'.h5')        

            indices_list.append(idx_labelled)
            inc = int(train.shape[0]*0.05 /  hyperparameters['batch_size'])
            num_batches+= inc
            print('Number of batches added:' , inc)       

            with open("./saved_history/history_list_"+filename+str(iteration)+".json", 'w') as f:
                json.dump(history_list, f, indent=2)

            with open("./saved_history/preds_"+filename+str(iteration)+".json", 'w') as f:
                json.dump(preds, f, indent=2)

            np.save("./saved_history/idx_labelled_"+filename+str(iteration)+".npy",np.array(idx_labelled))        

            if (iteration!=(len(splits)-1)): # last iteration
                k = random.sample(idx_unlabelled, hyperparameters['num_uncertain_elements'])
                idx_labelled = idx_labelled+list(k)
                labelled_train = pd.DataFrame(train.values[idx_labelled,:], columns=train.columns)
                idx_unlabelled = list(np.setdiff1d(idx_unlabelled, k))
                unlabelled_train = pd.DataFrame(train.values[idx_unlabelled,:], columns=train.columns)            


                train_gen = MultipleGenerator(labelled_train.values, 
                                              unlabelled_train.values, 
                                              labels,
                                              batch_size= hyperparameters['batch_size'],
                                              dim = (hyperparameters['height'],hyperparameters['width']),
                                              n_channels = hyperparameters['channels'],
                                              shuffle=True)

                unlabelled_gen = DataGenerator(unlabelled_train.values, 
                                       labels, 
                                       dim = (hyperparameters['height'],hyperparameters['width']),
                                       batch_size= hyperparameters['batch_size'],
                                       n_channels = hyperparameters['channels'],
                                       shuffle = False)
                labelled_gen = DataGenerator(labelled_train.values, 
                               labels, 
                               dim = (hyperparameters['height'],hyperparameters['width']),
                               batch_size= hyperparameters['batch_size'],
                               n_channels = hyperparameters['channels'],
                               shuffle = False)
                
    return history_list, pred_model, vae, disc, AL_model, indices_list, preds

In [32]:
attempt = '1'
cls = 'all'
filename = "Random_Sampling_CityScapes_" +cls + '_' + attempt

history_list, pred_model, vae, disc, Al_model, indices_list, preds = startTraining(filename, trackers, splits, break_point_ep, False, False, False, False, False)

model loaded
0 0.1 37
Epoch 1/7


/opt/conda/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


 1/37 [..............................] - ETA: 58:18 - loss_predictor_total: 1.4751 - acc_predictor: 0.4993 - loss_VAE: 0.1245 - loss_disc: 0.8275 - acc_disc: 0.3750 - acc_predictor_0: 0.5045 - IOU_predictor_0: 0.3373 - acc_predictor_1: 0.4980 - IOU_predictor_1: 0.3177 - acc_predictor_2: 0.4986 - IOU_predictor_2: 0.2539 - acc_predictor_3: 0.4989 - IOU_predictor_3: 0.2589 - acc_predictor_4: 0.4970 - IOU_predictor_4: 0.2799 - acc_predictor_5: 0.4988 - IOU_predictor_5: 0.2519 - acc_predictor_6: 0.4994 - IOU_predictor_6: 0.2737

KeyboardInterrupt: 

In [ ]:
# import os
# os.chdir(r'/kaggle/working/')
# from IPython.display import FileLink
# FileLink(r'./saved_history/training_results_Random_Sampling_CityScapes_all_1.csv')

In [ ]:
# # checking imbalance

# train_gen_full = DataGenerator(partition['train'], 
#                                    labels, 
#                                    dim = (hyperparameters['height'],hyperparameters['width']),
#                                    batch_size= hyperparameters['batch_size'],
#                                    n_channels = hyperparameters['channels'],
#                                    shuffle = True)

# val_gen = DataGenerator(partition['val'], 
#                                    labels, 
#                                    dim = (hyperparameters['height'],hyperparameters['width']),
#                                    batch_size= hyperparameters['batch_size'],
#                                    n_channels =hyperparameters['channels'],
#                                    shuffle = False)

# l1 = []
# t=0
# for i in train_gen_full:
#     l1.append(i[1])
#     t+=1
    
#     if t%100==0:
#         print(t)
        
# l2=[]
# t=0
# for i in val_gen:
#     l2.append(i[1])
#     t+=1
    
#     if t%100==0:
#         print(t)

In [ ]:
# p1 = [i[3] for i in l1]
# p2 =  [i[3] for i in l2]
# p1=np.concatenate(p1)
# p2=np.concatenate(p2)
# dic1={}
# for i in range(p1.shape[0]):
#     if i%500==0:
#         print(i)
#     for j in range(p1.shape[1]):
#         for k in range(p1.shape[2]):
#             if p1[i,j,k] in dic1:
#                 dic1[p1[i,j,k]]+=1
#             else:
#                 dic1[p1[i,j,k]]=1

# dic2={}
# for i in range(p2.shape[0]):
#     if i%500==0:
#         print(i)
#     for j in range(p2.shape[1]):
#         for k in range(p2.shape[2]):
#             if p2[i,j,k] in dic2:
#                 dic2[p2[i,j,k]]+=1
#             else:
#                 dic2[p2[i,j,k]]=1

In [ ]:
# {1: 48034810, 0: 49220614} {1: 8467216, 0: 7785712} - 
# {0: 76243374, 1: 21012050} {0: 13708944, 1: 2543984}
# {0: 95544125, 1: 1711299} {0: 16086768, 1: 166160}
# {0: 93785234, 1: 3470190} {0: 16090240, 1: 162688}
# {0: 82503746, 1: 14751678}, {0: 13721840, 1: 2531088} - 
# {0: 96086664, 1: 1168760} {0: 16184976, 1: 67952}   - 
# {0: 90148787, 1: 7106637} {0: 13939088, 1: 2313840} - 

In [ ]:
# r = [{1: 48034810, 0: 49220614},
#  {0: 76243374, 1: 21012050},
#  {0: 95544125, 1: 1711299},
#  {0: 93785234, 1: 3470190},
#  {0: 82503746, 1: 14751678},
#  {0: 96086664, 1: 1168760},
#  {0: 90148787, 1: 7106637}]

# wts = []
# for i in r:
#     wts.append([((i[0]+i[1])/(2*i[0])), ((i[0]+i[1])/(2*i[1]))])

In [ ]:
# tot1=0
# for i in dic1:
#     tot1+=dic1[i]
    
# for i in dic1:
#     print(i, (dic1[0]+dic1[1])/(2*dic1[i]))

In [ ]:
# tot1=0
# for i in dic1:
#     tot1+=dic1[i]

# for i in dic1:
#     print(i, (dic1[i])/tot1)

In [ ]:
# tot2=0
# for i in dic2:
#     tot2+=dic2[i]

# for i in dic2:
#     print(i, (dic2[i])/tot2)

## Individual testing

In [ ]:
# tf.config.run_functions_eagerly(True)
# attempt = '1'
# cls = 'flat'
# filename = "Random_Sampling_CityScapes_" +cls + '_' + attempt

# preds=[]
# validation_train_history=[] # new
# logdir = "./logs/" + filename
# filepath = "./saved_history/model/"+filename

# checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=False, mode='max', period = 1)
# csv_logger = CSVLogger('./saved_history/training_results_'+filename+'.csv', separator = ',', append=True)
# pre_train_logger = CSVLogger('./saved_history/pre_training_results_'+filename+'.csv', separator = ',', append=True)
# tensorboard_callback = TensorBoard(log_dir = logdir)
# pre_tensorboard_callback = TensorBoard(log_dir =" ./logs/pre_"+filename)

# # predictor
# pred_model = vgg_segnet(n_classes = hyperparameters['num_tasks'], input_height=hyperparameters['height'], input_width=hyperparameters['width'], encoder_level=2, channels=hyperparameters['channels'])
# pred_model.compile(optimizer = keras.optimizers.SGD(learning_rate=hyperparameters['lr'],
#                                                 clipnorm=1.0 ))

# # defining my discriminator
# disc_in, disc_out = discriminator()
# disc = Model(inputs = disc_in, outputs = disc_out)
# disc.compile(optimizer = keras.optimizers.SGD(learning_rate=hyperparameters['lr'],
#                                               clipnorm=1.0 ))

# # defining my generator
# X, z, mu = variationalAutoEncoder()
# vae = Model(inputs = X, outputs = [z,mu])
# vae.compile(optimizer = keras.optimizers.RMSprop(learning_rate=hyperparameters['lr'],
#                                                  clipnorm=1.0 ))

# # Instantiate AL model
# AL_model = ActiveLearning(discriminator=disc, generator=vae, predictor=pred_model, hyperparameters=hyperparameters, trackers = trackers)
# AL_model.compile(
#     d_optimizer=keras.optimizers.SGD(learning_rate=hyperparameters['lr'],clipnorm=1.0 ),
#     g_optimizer= keras.optimizers.RMSprop(learning_rate=hyperparameters['lr'],clipnorm=1.0),
#     p_optimizer=keras.optimizers.SGD(learning_rate=hyperparameters['lr'],clipnorm=1.0 ))

# print('model loaded')

# (train, val) = preprocess(hyperparameters)

# # Generators
# train_gen_full = DataGenerator(partition['train'], 
#                                    labels, 
#                                    dim = (hyperparameters['height'],hyperparameters['width']),
#                                    batch_size= hyperparameters['batch_size'],
#                                    n_channels = hyperparameters['channels'],
#                                    shuffle = True)

# val_gen = DataGenerator(partition['val'], 
#                                    labels, 
#                                    dim = (hyperparameters['height'],hyperparameters['width']),
#                                    batch_size= hyperparameters['batch_size'],
#                                    n_channels =hyperparameters['channels'],
#                                    shuffle = False)

In [ ]:
# checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=False, mode='max', period = 1)
# csv_logger = CSVLogger('./saved_history/training_results_'+filename+'.csv', separator = ',', append=True)
# pre_train_logger = CSVLogger('./saved_history/pre_training_results_'+filename+'.csv', separator = ',', append=True)
# tensorboard_callback = TensorBoard(log_dir = logdir)
# pre_tensorboard_callback = TensorBoard(log_dir =" ./logs/pre_"+filename)

# # predictor
# pred_model = vgg_segnet(n_classes = hyperparameters['num_tasks'], input_height=hyperparameters['height'], input_width=hyperparameters['width'], encoder_level=2, channels=hyperparameters['channels'])
# pred_model.compile(optimizer = keras.optimizers.SGD(learning_rate=hyperparameters['lr'],
#                                                 clipnorm=1.0 ))

# # defining my discriminator
# disc_in, disc_out = discriminator()
# disc = Model(inputs = disc_in, outputs = disc_out)
# disc.compile(optimizer = keras.optimizers.SGD(learning_rate=hyperparameters['lr'],
#                                               clipnorm=1.0 ))

# # defining my generator
# X, z, mu = variationalAutoEncoder()
# vae = Model(inputs = X, outputs = [z,mu])
# vae.compile(optimizer = keras.optimizers.RMSprop(learning_rate=hyperparameters['lr'],
#                                                  clipnorm=1.0 ))

# # Instantiate AL model
# AL_model = ActiveLearning(discriminator=disc, generator=vae, predictor=pred_model, hyperparameters=hyperparameters, trackers = trackers)
# AL_model.compile(
#     d_optimizer=keras.optimizers.SGD(learning_rate=hyperparameters['lr'],clipnorm=1.0 ),
#     g_optimizer= keras.optimizers.RMSprop(learning_rate=hyperparameters['lr'],clipnorm=1.0),
#     p_optimizer=keras.optimizers.SGD(learning_rate=hyperparameters['lr'],clipnorm=1.0 ))

# print('model loaded')

In [ ]:
# p_optimizer=keras.optimizers.SGD(learning_rate=hyperparameters['lr'],clipnorm=1.0 )

In [ ]:
# for i in train_gen_full:
#     k=i
#     break

In [ ]:
# labelled_x=k[0]
# labelled_y=k[1]

In [ ]:
# def dice_coef(y_true, y_pred):
#     smooth = 1.0
#     y_true_f = tf.cast(K.flatten(y_true), tf.float64)
#     y_pred_f = tf.cast(K.flatten(y_pred), tf.float64)
#     intersection = K.sum(y_true_f * y_pred_f)
#     return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# def dice_coef_loss(y_true, y_pred):
#     t = []
#     for i in range(7):
#         t.append(1 - dice_coef(y_true[i], y_pred[i]))
        
#     return tf.convert_to_tensor(t)

In [ ]:
# with tf.GradientTape() as tape:
#     labelled_prediction_y,_ = pred_model(labelled_x, training=True)
    
#     re_1=[]
#     for i in range(7):
#         re_1.append(tf.reshape(labelled_y[i], (hyperparameters['batch_size'], -1)))

#     re_2 = []
#     for i in range(7):
#         re_2.append(tf.reshape(labelled_prediction_y[i], (hyperparameters['batch_size'], -1)))
        
#     weights = [{1: 1.0123, 0: 0.9879},
#      {0: 0.6377, 1: 2.3142},
#      {0: 0.5089, 1: 28.4156},
#      {0: 0.5185, 1: 14.0129},
#      {0: 0.5894, 1: 3.2964},
#      {0: 0.5060, 1: 41.6062},
#      {0: 0.5394, 1: 6.8425}]
    
#     w = []
#     for i in range(7):
#         wts = np.zeros((re_1[i].shape))
#         for j in range(re_1[i].shape[0]):
#             t = re_1[i][j].numpy()
#             t = t.astype('float64')
#             wts[j][t==0] = weights[i][0]
#             wts[j][t==1] = weights[i][1]

#         wts=tf.convert_to_tensor(wts)
#         predictor_loss = weighted_cross_entropy(re_1[i], re_2[i], sample_weights=wts)
#         w.append(predictor_loss)
    
#     w = tf.convert_to_tensor(w)
#     w = tf.math.reduce_mean(w, axis=1) +  tf.cast(dice_coef_loss(re_1, re_2), tf.float32)

# trainable_vars = pred_model.trainable_variables
# gradients = tape.gradient(w, trainable_vars)

# # Update weights
# p_optimizer.apply_gradients(zip(gradients, trainable_vars))        

In [ ]:
# with tf.GradientTape() as tape:
#     labelled_prediction_y,_ = pred_model(labelled_x, training=True)
    
#     re_1=[]
#     for i in range(7):
#         re_1.append(tf.reshape(labelled_y[i], (hyperparameters['batch_size'], -1)))

#     re_2 = []
#     for i in range(7):
#         re_2.append(tf.reshape(labelled_prediction_y[i], (hyperparameters['batch_size'], -1)))
    
# #     w = []
# #     for i in range(7):
# #         wts = np.zeros((re_1[i].shape))
# #         for j in range(re_1[i].shape[0]):
# #             t = re_1[i][j].numpy()
# #             t = t.astype('float64')
# #             wts[j][t==0] = weights[i][0]
# #             wts[j][t==1] = weights[i][1]

# #         wts=tf.convert_to_tensor(wts)
# #         predictor_loss = weighted_cross_entropy(re_1[i], re_2[i], sample_weights=wts)
# #         w.append(predictor_loss)
# #     w = tf.convert_to_tensor(w)
# #     w = tf.math.reduce_mean(w, axis=1)
    
# # trainable_vars = pred_model.trainable_variables
# # gradients = tape.gradient(w, trainable_vars)

# # # Update weights
# # p_optimizer.apply_gradients(zip(gradients, trainable_vars))        

In [ ]:
# def dice_coef(y_true, y_pred):
#     smooth = 1.0
#     y_true_f = tf.cast(K.flatten(y_true), tf.float64)
#     y_pred_f = tf.cast(K.flatten(y_pred), tf.float64)
#     intersection = K.sum(y_true_f * y_pred_f)
#     return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# def dice_coef_loss(y_true, y_pred):
#     t = []
#     for i in range(7):
#         t.append(1 - dice_coef(y_true[i], y_pred[i]))
        
#     return tf.convert_to_tensor(t)

In [ ]:
# l = tfa.losses.SigmoidFocalCrossEntropy()
# l(re_1, re_2)

In [ ]:
# def weighted_cross_entropy(y_true, y_pred, sample_weights):
#     y_true = tf.cast(y_true,y_pred.dtype)
#     q1 = -1*(tf.math.multiply(y_true,tf.math.log(y_pred)) + tf.math.multiply((1.0-y_true),tf.math.log(1.0-y_pred)))
#     sample_weights = tf.cast(sample_weights,q1.dtype)
#     z = tf.math.multiply(q1,sample_weights)
    
#     return tf.math.reduce_mean(z,axis=1)

In [ ]:
# weights = [{1: 1.0123, 0: 0.9879},
#  {0: 0.6377, 1: 2.3142},
#  {0: 0.5089, 1: 28.4156},
#  {0: 0.5185, 1: 14.0129},
#  {0: 0.5894, 1: 3.2964},
#  {0: 0.5060, 1: 41.6062},
#  {0: 0.5394, 1: 6.8425}]

# w = []
# for i in range(7):
#     wts = np.zeros((re_1[i].shape))
#     for j in range(re_1[i].shape[0]):
#         t = re_1[i][j].numpy()
#         t = t.astype('float64')
#         wts[j][t==0] = weights[i][0]
#         wts[j][t==1] = weights[i][1]
        
#     wts=tf.convert_to_tensor(wts)
#     predictor_loss = weighted_cross_entropy(re_1[i], re_2[i], sample_weights=wts)
#     w.append(predictor_loss)
#     w = tf.convert_to_tensor(w)
#     tf.math.reduce_mean(w, axis=1)

In [ ]:
# a = tf.convert_to_tensor([[0.1,0.2,0.7], [0.4,0.3,0.5]])
# y = tf.convert_to_tensor([[0.2,0.2,0.6], [0.1,0.1,0.8]])
# sample_weights = tf.convert_to_tensor([[1,2,3], [2,4,5]])
# a1 = weighted_cross_entropy(a,y,sample_weights)

In [ ]:
# iou=tf.keras.metrics.MeanIoU(num_classes=2)
# iou(re_1[1], tf.cast(re_2[1]>=0.5, tf.int64))